# Data Loading and Processing

* pip install pyarrow  

## 1. Imports and setup

In [1]:
# !wget https://raw.githubusercontent.com/benhamner/Metrics/master/Python/ml_metrics/average_precision.py

In [2]:
# helper functions
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from average_precision import apk, mapk

# Helper functions for improving memory
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

# Transforms categorical columns into categorical types
class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        ''' Identifies the categorical variables in our data based on the number of unique value counts'''
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts() # value counts for each column
            self.categories.append(vc[vc > self.min_examples].index.tolist()) # keeps columns whose value counts are large enough
        return self

    def transform(self, X):
        ''' Transforms the categorical variables into numerical codes'''
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        print(data)
        return pd.DataFrame(data=data)

## 2. Read Data

In [3]:
import pandas as pd

transactions = pd.read_csv('data/transactions_train.csv', dtype={"article_id": "str"})
customers = pd.read_csv('data/customers.csv')
articles = pd.read_csv('data/articles.csv', dtype={"article_id": "str"})

## 3. Process Data

#### Apply memory reducing techniques to transactions.csv

In [4]:
# Convert customer_id to int
transactions['customer_id'] = customer_hex_id_to_int(transactions['customer_id'])
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       uint64 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(2), uint64(1)
memory usage: 4.7 GB


In [5]:
transactions.isna().sum()

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64

In [6]:
#Remove rows with NAs
transactions.dropna(inplace=True)

In [7]:
# Convert transaction dates to datetime
transactions.t_dat = pd.to_datetime(transactions.t_dat, format='%Y-%m-%d')

In [8]:
# Convert article_id, price, sales_channel_id, and week
transactions.article_id = article_id_str_to_int(transactions.article_id)
articles.article_id = article_id_str_to_int(articles.article_id)

transactions.sales_channel_id = transactions.sales_channel_id.astype('int8')
transactions.price = transactions.price.astype('float32')

#### Add *week* in transactions table

Week 0 corresponds to the first week and week 104 corresponds to the final week of the data

In [9]:
DURATION_IN_WEEKS = 104 # Transactions take place over 104 weeks
transactions['week'] = DURATION_IN_WEEKS - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7

# Memory optimization
transactions.week = transactions.week.astype('int8')

In [10]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       uint64        
 2   article_id        int32         
 3   price             float32       
 4   sales_channel_id  int8          
 5   week              int8          
dtypes: datetime64[ns](1), float32(1), int32(1), int8(2), uint64(1)
memory usage: 1.0 GB


#### Apply memory reducing techniques to customers.csv

In [11]:
customers.customer_id = customer_hex_id_to_int(customers.customer_id)

In [12]:
customers.isna().sum()

customer_id                    0
FN                        895050
Active                    907576
club_member_status          6062
fashion_news_frequency     16009
age                        15861
postal_code                    0
dtype: int64

In [13]:
#Drop FN and Active Columns
customers.drop(["Active", "FN"], axis=1, inplace=True)

In [14]:
#Replace Age with the mean
mean_age = customers['age'].median()
customers['age'].fillna(mean_age,inplace=True)

In [15]:
#For club_member_status and fashion_news_frequency replace NAs with most common value
customers['fashion_news_frequency'].fillna(customers['fashion_news_frequency'].value_counts().index[0],inplace=True)
customers['club_member_status'].fillna(customers['club_member_status'].value_counts().index[0],inplace=True)

In [16]:
customers.club_member_status = Categorize().fit_transform(customers[['club_member_status']]).club_member_status
customers.postal_code = Categorize().fit_transform(customers[['postal_code']]).postal_code
customers.fashion_news_frequency = Categorize().fit_transform(customers[['fashion_news_frequency']]).fashion_news_frequency

{'club_member_status': array([0, 0, 0, ..., 0, 0, 1], dtype=int8)}
{'postal_code': array([  6305,  33726,   3247, ..., 106737, 111894, 352898], dtype=int32)}
{'fashion_news_frequency': array([0, 0, 0, ..., 1, 1, 0], dtype=int8)}


#### Apply memory reducing techniques to articles.csv

In [17]:
for col in articles.columns:
    if articles[col].dtype == 'object':
        articles[col] = Categorize().fit_transform(articles[[col]])[col]

{'prod_name': array([12855, 12855, 44846, ..., 24040, 24041, 45874], dtype=int32)}
{'product_type_name': array([ 9,  9,  9, ...,  1, 48,  1], dtype=int16)}
{'product_group_name': array([0, 0, 0, ..., 2, 3, 2], dtype=int8)}
{'graphical_appearance_name': array([0, 0, 3, ..., 0, 0, 0], dtype=int8)}
{'colour_group_name': array([ 0,  2, 11, ...,  0,  0, 11], dtype=int8)}
{'perceived_colour_value_name': array([0, 2, 1, ..., 0, 0, 1], dtype=int8)}
{'perceived_colour_master_name': array([0, 2, 2, ..., 0, 0, 2], dtype=int8)}
{'department_name': array([10, 10, 10, ...,  0, 29,  0], dtype=int16)}
{'index_code': array([0, 0, 0, ..., 0, 1, 0], dtype=int8)}
{'index_name': array([0, 0, 0, ..., 0, 1, 0], dtype=int8)}
{'index_group_name': array([0, 0, 0, ..., 0, 2, 0], dtype=int8)}
{'section_name': array([30, 30, 30, ..., 12, 25, 12], dtype=int8)}
{'garment_group_name': array([2, 2, 2, ..., 0, 1, 0], dtype=int8)}
{'detail_desc': array([ 8834,  8834,  8834, ..., 28024, 28023, 43403], dtype=int32)}


In [18]:
for col in articles.columns:
    if articles[col].dtype == 'int64':
        articles[col] = articles[col].astype('int32')

## 4. Save preprocessed data in parquet files

In [19]:
transactions.sort_values(['t_dat', 'customer_id'], inplace=True)

In [20]:
transactions.to_parquet('data/transactions_train.parquet')
customers.to_parquet('data/customers.parquet')
articles.to_parquet('data/articles.parquet')

## 5. Create Training and Test sets

"Leave One Last Item" train, validation, test split from https://arxiv.org/pdf/2007.13237.pdf

In [21]:
val_week = transactions["week"].max() # Last week in the training set

# train/ val transactions sets
transactions_train = transactions[transactions["week"] != val_week].reset_index(drop = True)
transactions_val = transactions[transactions["week"] == val_week].reset_index(drop = True)

In [22]:
transactions_train.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week
0,2018-09-20,1728846800780188,519773001,0.028458,2,0
1,2018-09-20,1728846800780188,578472001,0.032525,2,0
2,2018-09-20,2076973761519164,661795002,0.167797,2,0
3,2018-09-20,2076973761519164,684080003,0.101678,2,0
4,2018-09-20,2918879973994241,662980001,0.033881,1,0


## 6. Create Training "Targets" (Ground Truth)

ALL WEEKS - LAST WEEK

Ground Truth: a dictionary of all customers and the list of articles they purchased in the training weeks

In [23]:
from collections import defaultdict

train_ground_truth = transactions_train\
        .groupby('customer_id')['article_id']\
        .apply(list)\
        .to_dict()

save dictionary as pkl file

In [24]:
pd.to_pickle(dict(train_ground_truth), 'data/train_ground_truth.pkl')

## 7. Create Validation "Targets" (Ground Truth) 

LAST WEEK

Ground Truth: a dictionary of all customers and the list of articles they purchased in the last week

In [25]:
from collections import defaultdict

val_ground_truth = transactions_val\
    .groupby("customer_id")["article_id"]\
    .apply(list)\
    .to_dict()

# example format of val_week_purchases_by_cust
#val_ground_truth[1402273113592184] # key = 1402273113592184

save dictionary as pkl file

In [26]:
pd.to_pickle(dict(val_ground_truth), 'data/val_ground_truth.pkl')

## 8. Subset Trainig Data

For subseting the data we are performing two steps:
1. Randomly subset a percentage of the rows from the transactions dataset
2. Retain only the customers with more than k transactions

### 8.1 Random selection

In [27]:
len(customers)

1371980

In [28]:
# Create a 3% sample of the entiriety of the data to speed up dev
sample = 0.03
customers_sample_rnd = customers.sample(frac=sample, replace=False, random_state = 42)
customers_sample_ids = set(customers_sample_rnd['customer_id'])
transactions_sample_rnd = transactions_train[transactions_train["customer_id"].isin(customers_sample_ids)]
articles_sample_ids = set(transactions_sample_rnd["article_id"])
articles_sample_rnd = articles[articles["article_id"].isin(articles_sample_ids)]

len(customers_sample_rnd)

41159

In [29]:
transactions_sample_rnd.shape

(934980, 6)

### 8.2 Retain customers with enough history data

Identify customers with more than *k* transactions

In [30]:
MIN_TRANSACTIONS = 15

# Retain only customers who make at least MIN_TRANSACTIONS transactions
num_transactions = transactions_sample_rnd["customer_id"].value_counts()
customer_ids_retained = num_transactions[num_transactions > MIN_TRANSACTIONS].index.tolist()

len(customer_ids_retained)

15334

Subsample customers, transactions, and articles

In [31]:
# Subsample of customers, transactions
customers_sample = customers_sample_rnd[customers_sample_rnd["customer_id"].isin(customer_ids_retained)]
transactions_sample = transactions_sample_rnd[transactions_sample_rnd["customer_id"].isin(customer_ids_retained)]

# Subsample of articles
article_ids_retained = set(transactions_sample_rnd["article_id"])
articles_sample = articles_sample_rnd[articles_sample_rnd["article_id"].isin(article_ids_retained)]

transactions_sample.shape

(796281, 6)

## 9. Save Training and Validation Data

Save Training data into parquets:

In [32]:
customers_sample.to_parquet(f'data/customers_train_sample_gt{MIN_TRANSACTIONS}transactions.parquet', index=False)
transactions_sample.to_parquet(f'data/transactions_train_sample_gt{MIN_TRANSACTIONS}transactions.parquet', index=False)
articles_sample.to_parquet(f'data/articles_train_sample_gt{MIN_TRANSACTIONS}transactions.parquet', index=False)

Save Validation data into parquets:

In [33]:
transactions_val.to_parquet(f'data/transactions_val.parquet', index=False)